In [42]:
import bs4
import requests
import pandas as pd

In [43]:

def scrape_listings(url):
    rows = []
    r = requests.get(url)

    soup = bs4.BeautifulSoup(r.text, 'html.parser')

    listings = soup.find_all('div', {'class': 'result-item'})
    for element in listings:
        row = {}
        row['url'] = element.find('a').get('href')

        url = 'https://www.exchangeandmart.co.uk' + row['url']
        det_url = requests.get(url)
        soup = bs4.BeautifulSoup(det_url.text, 'html.parser')
        name = soup.find('span', {'class': 'ttl'})
        model = soup.find('span', {'class': 'ttlSup'})
        price = soup.find('span', {'itemprop': 'price'})

        try:
            row['name'] = str(name.text).strip()
            row['model'] = str(model.text).strip()
            row['price'] = str(price.text).strip()
        except:
            print("name, model or price not found! continuing...")
            continue

        facts = element.find('div', {'class': 'key-details'})
        sayi = 1
        for fact in facts.find_all('p'):
            if(sayi != 4 and sayi != 5):
                row[sayi] = str(fact.text).strip()
            sayi+=1
            #add model and name to rows
        
        det1 = soup.find('div', {'class': 'adDetails'})
        det2 = soup.find('div', {'class': 'adDetsColumn'})

        for cell in det1.find_all('div', {'class': 'adDetsItem'}):
            row[sayi] = str(cell.text).strip()
            sayi+=1
        counter = 1
        
        try:
            for span in det2.find_all('span'):
                if counter%2 == 0:
                    row[sayi] = str(span.text).strip()
                    sayi+=1
                counter        

            rows.append(row)
        except:
            print("details not found! continuing...")
            continue

    return pd.DataFrame(rows)


In [ ]:
# Solution
df = scrape_listings('https://www.exchangeandmart.co.uk/used-cars-for-sale/mercedes_benz/')
df

In [45]:
def scrape_all(pages):
    url = "https://www.exchangeandmart.co.uk/used-cars-for-sale/mercedes_benz/"
    sort_filter = "?sort=price"
    df = pd.DataFrame()
    for page in pages:
        df = pd.concat([df, scrape_listings(url + "page" + str(page) + sort_filter)], ignore_index=True)
    return df

In [ ]:
df_large = scrape_all(range(1, 101))
df_large

In [47]:
df_large.to_csv('file_name.csv', index=False)